<a href="https://colab.research.google.com/github/ozakiryota/cart_pole/blob/main/cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cart-Pole


Install OpenAI Gym (https://gym.openai.com)

In [1]:
!pip install gym

Install the packages for visualizing Gym

In [2]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

## Random move

Import Gym

In [3]:
import gym

Import the packages for visualizing Gym

In [4]:
import base64
import io
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

Move the cart-pole randomly

In [5]:
d = Display()
d.start()

env = Monitor(gym.make('CartPole-v0'),'./videos/random_move/', force=True)
print("env.observation_space.shape = ", env.observation_space.shape)
print("env.action_space.n = ", env.action_space.n)

obs = env.reset()

for t in range(100):
    obs, reward, is_done, info = env.step(env.action_space.sample())
    print("obs = ", obs)
    print("reward = ", reward)
    print("info = ", info)

    if is_done:
        print("Episode finished after {} timesteps".format(t+1))
        env.reset()
        break

for frame in env.videos:
    print("frame = ", frame)
    video = io.open(frame[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))

env.observation_space.shape =  (4,)
env.action_space.n =  2
obs =  [ 0.01932852  0.15622655 -0.03086705 -0.35115304]
reward =  1.0
info =  {}
obs =  [ 0.02245305  0.35177356 -0.03789011 -0.6534073 ]
reward =  1.0
info =  {}
obs =  [ 0.02948852  0.15719914 -0.05095825 -0.37289217]
reward =  1.0
info =  {}
obs =  [ 0.03263251 -0.03716327 -0.0584161  -0.09670225]
reward =  1.0
info =  {}
obs =  [ 0.03188924  0.15874515 -0.06035014 -0.4072282 ]
reward =  1.0
info =  {}
obs =  [ 0.03506414 -0.03547145 -0.06849471 -0.13416583]
reward =  1.0
info =  {}
obs =  [ 0.03435472 -0.22954884 -0.07117802  0.13614593]
reward =  1.0
info =  {}
obs =  [ 0.02976374 -0.03348338 -0.0684551  -0.17811625]
reward =  1.0
info =  {}
obs =  [ 0.02909407  0.16254802 -0.07201743 -0.49158475]
reward =  1.0
info =  {}
obs =  [ 0.03234503 -0.03148812 -0.08184912 -0.22243982]
reward =  1.0
info =  {}
obs =  [ 0.03171527  0.16470253 -0.08629792 -0.53977737]
reward =  1.0
info =  {}
obs =  [ 0.03500932 -0.02910707 -0.097

## Q-learning

Import required packages

In [6]:
import numpy as np

Brain class

In [12]:
class Brain:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.num_states = num_states
        self.list_state_range = list_state_range
        self.list_state_reso = list_state_reso
        self.num_actions = num_actions

        self.eps = 1.0  # for epsilon greedy algorithm
        self.gamma = gamma
        self.r = r
        self.lr = lr

        self.q_table = np.random.rand(np.prod(list_state_reso), num_actions)

    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]
 
    def getStateIndex(self, observation):
        list_index = []
        for i in range(self.num_states):
            index = np.digitize(observation[i], bins=self.bins(self.list_state_range[i][0], self.list_state_range[i][1], self.list_state_reso[i]))
            list_index.append(index)
        return sum([index*int(np.prod(self.list_state_reso[:i])) for i, index in enumerate(list_index)])

    def updateQtable(self, obs, action, reward, next_obs):
        q = self.q_table[self.getStateIndex(obs), action]
        next_q_max = np.max(self.q_table[self.getStateIndex(next_obs)])
        self.q_table[self.getStateIndex(obs), action] = q + self.lr*(reward + self.gamma*next_q_max - q)

    def getAction(self, obs, is_training):
        if is_training and np.random.rand() < self.eps:
            action = np.random.randint(self.num_actions)
        else:
            action = np.argmax(self.q_table[self.getStateIndex(obs)])
        ## update eps
        if is_training and self.eps > 0.1:
            self.eps *= self.r
        return action

Agent class

In [8]:
class Agent:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.brain = Brain(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def updateQtable(self, obs, action, reward, next_obs):
        self.brain.updateQtable(obs, action, reward, next_obs)
 
    def getAction(self, obs, is_training):
        action = self.brain.getAction(obs, is_training)
        return action

Environment class

In [9]:
class Environment:
    def __init__(self, num_episodes, max_step, gamma, r, lr):
        ## parameters
        self.num_episodes = num_episodes
        self.max_step = max_step
        ## environment
        self.env = Monitor(gym.make('CartPole-v0'), './videos/q_learning/', force=True)
        ## agent
        num_states = self.env.observation_space.shape[0]    # position, velocity, angle, angular velocity
        list_state_range = []
        for i in range(num_states):
            list_state_range.append([env.observation_space.low[i], env.observation_space.high[i]])
        list_state_range[1] = [-3.0, 3.0]
        list_state_range[3] = [-0.5, 0.5]
        print("list_state_range = ", list_state_range)
        list_state_reso = [4, 4, 6, 6]
        num_actions = self.env.action_space.n

        self.agent = Agent(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def train(self):
        num_completed_episodes = 0
  
        for episode in range(self.num_episodes):
            obs = self.env.reset()
            episode_reward = 0
 
            for step in range(self.max_step):
                ## get action
                action = self.agent.getAction(obs, is_training=True)
                ## observe next step
                next_obs, _, is_done, _ = self.env.step(action)
                ## get reward
                if is_done:
                    if step < max_step - 1:
                        reward = -100
                    else:
                        reward = 1
                        num_completed_episodes += 1
                else:
                    reward = 1
                episode_reward += reward
                ## update
                self.agent.updateQtable(obs, action, reward, next_obs)
                ## to next step
                obs = next_obs

                if is_done:
                    print('{0} Episode: Finished after {1} time steps with reward {2}'.format(episode, step+1, episode_reward))
                    break
        print("num_completed_episodes = ", num_completed_episodes)

    def evaluate(self):
        obs = self.env.reset()
        
        for step in range(self.max_step):
            ## get action
            action = self.agent.getAction(obs, is_training=False)
            ## observe next step
            next_obs, _, is_done, _ = self.env.step(action)
            ## to next step
            obs = next_obs

            if is_done:
                print('Evaluation: Finished after {} time steps'.format(step+1))
                break

Prepare showing videos of the restults

In [10]:
def show_video(env):
    for frame in env.videos:
        print("frame = ", frame)
        video = io.open(frame[0], 'r+b').read()
        encoded = base64.b64encode(video)

        display.display(display.HTML(data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>
            """.format(encoded.decode('ascii')))
        )

Run training

In [ ]:
## parameters
num_episodes = 500
max_step = 200
gamma = 0.9
r = 0.99
lr = 0.5

## run
cartpole_env = Environment(num_episodes, max_step, gamma, r, lr)
cartpole_env.train()
print("Evaluation")
cartpole_env.evaluate()
show_video(cartpole_env.env)

list_state_range =  [[-4.8, 4.8], [-3.0, 3.0], [-0.41887903, 0.41887903], [-0.5, 0.5]]
0 Episode: Finished after 30 time steps with reward -71
1 Episode: Finished after 15 time steps with reward -86
2 Episode: Finished after 10 time steps with reward -91
3 Episode: Finished after 12 time steps with reward -89
4 Episode: Finished after 16 time steps with reward -85
5 Episode: Finished after 14 time steps with reward -87
6 Episode: Finished after 16 time steps with reward -85
7 Episode: Finished after 16 time steps with reward -85
8 Episode: Finished after 40 time steps with reward -61
9 Episode: Finished after 9 time steps with reward -92
10 Episode: Finished after 68 time steps with reward -33
11 Episode: Finished after 39 time steps with reward -62
12 Episode: Finished after 34 time steps with reward -67
13 Episode: Finished after 10 time steps with reward -91
14 Episode: Finished after 49 time steps with reward -52
15 Episode: Finished after 31 time steps with reward -70
16 Episode: 

## References
- [Getting Started with Gym](https://gym.openai.com/docs/)
- [【強化学習】OpenAI Gym を Google Colab上で描画する方法 (2020.6版)](https://qiita.com/ymd_h/items/c393797deb72e1779269)
- [minnano_rl/section_2/01_simple_reinforcement_learning.ipynb](https://github.com/yukinaga/minnano_rl/blob/main/section_2/01_simple_reinforcement_learning.ipynb)
- [第10回　CartPole課題をQ学習で制御する](https://book.mynavi.jp/manatee/detail/id=88997)